# SAMap zebrafish Farrell vs amphioxus results
With leiden-based neighborhoods

Zebrafish proteins

In [ ]:
from samap.mapping import SAMAP
from samap.analysis import (get_mapping_scores, GenePairFinder, sankey_plot, chord_plot, CellTypeTriangles, 
                            ParalogSubstitutions, FunctionalEnrichment,
                            convert_eggnog_to_homologs, GeneTriangles)
from samalg import SAM
import pandas as pd
import numpy
import seaborn as sns
import csv
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context
import scanpy as sc
import warnings
warnings.filterwarnings(action='ignore')
import pickle
import anndata as ad
import holoviews as hv

with open('sam_processed.pkl', 'rb') as inp:
    sm = pickle.load(inp)

## Read parameters

In [ ]:
with open('config.csv') as f:
    config=[tuple(line) for line in csv.reader(f)]

config=dict(config)
contrast='farrell_'+config["stage_zebra"]+'_vs_'+config["stage_amp"]
print(contrast)

Show parameters.

In [ ]:
pd.DataFrame.from_dict(config,orient='index',dtype='str')

## Add coarse annotation

In [ ]:
with open('../../input_data/farell_coarse_annot_map.csv') as f:
    annot_map=[tuple(line) for line in csv.reader(f)]

annot_map=dict(annot_map)

sm.sams['dr'].adata.obs['celltype_coarse']=sm.sams['dr'].adata.obs['segment_celltype'].replace(annot_map).astype(str)

sm.sams['dr'].adata.obs['celltype_coarse_stage']=sm.sams['dr'].adata.obs['celltype_coarse'].astype(str)+'_'+sm.sams['dr'].adata.obs['stage_fine'].astype(str)

concat_ads=ad.concat({'Zebrafish':sm.sams['dr'].adata,
                     'Amphioxus':sm.sams['bf'].adata}, label="dataset")

# Post-SAMap analysis

## Visualizing the combined projection

In [ ]:
sc.pl.embedding(concat_ads,
                    basis='X_umap_samap',
                    color='dataset',
                frameon=False,
                title='SAMap UMAP amphioxus '+config["stage_amp"]+' vs Zebrafish '+config["stage_zebra"],
                palette={'Zebrafish':('#1b9e77',0.8),'Amphioxus':('#d95f02',0.8)})

Visualise both objects with celltypes

In [ ]:
pal_bf={'Ectoderm':'tab:blue',
       'Endoderm':'tab:orange',
       'Mesoderm':'tab:green',
       'Neural':'tab:red',
       'Notochord':'tab:purple',
       'Unidentified':'tab:cyan'}

pal_dr={'Ectoderm':'tab:blue',
       'Endoderm':'tab:orange',
       'Mesoderm':'tab:green',
       'Neural':'tab:red',
       'Notochord':'tab:purple',
       'Unassigned_types':'tab:cyan'}
sc.pl.embedding(sm.sams['bf'].adata,
                    basis='X_umap_samap',
                    color='celltype_coarse2',
                title=('Amphioxus integrated '+config["stage_amp"]),
                    frameon=False,save='_amp'+config["stage_amp"]+'_int_celltypes.pdf',
               palette=pal_bf)
sc.pl.embedding(sm.sams['dr'].adata,
                    basis='X_umap_samap',
                title=('Zebrafish integrated '+config["stage_zebra"]),
                    color='celltype_coarse',
                    frameon=False,save='_dr'+config["stage_zebra"]+'_int_celltypes.pdf',
               palette=pal_dr)

## Selected celltype visualisation

In [ ]:
ncols = 3
nrows = 1
figsize =6.5
wspace = 0.3
fig, axs = plt.subplots(
    nrows=nrows,
    ncols=ncols,
    figsize=(ncols * figsize + figsize * wspace * (ncols - 1), nrows * figsize),
)
plt.subplots_adjust(wspace=wspace)

sc.pl.embedding(sm.sams['bf'].adata,
                    basis='X_umap_samap',
                    color='celltype_fine',
                    groups=['PrechordalPlate','Notochord','MNCC-likeCells','PMNCC'],
                    frameon=True,
                   title='Amphioxus '+config["stage_amp"],ax=axs[0], show=False)
#sm.scatter(COLORS={'dr':'#10b9f1','bf':'#f14810'},axes=axs[1],ss={'bf':5,'dr':5},alpha=0.5)

sc.pl.embedding(concat_ads,
                    basis='X_umap_samap',
                    color='dataset',
                frameon=True,
                title='Combined by SAMap',
                palette={'Zebrafish':('#1b9e77',0.8),'Amphioxus':('#d95f02',0.8)},ax=axs[1],show=False)

sc.pl.embedding(sm.sams['dr'].adata,
                    basis='X_umap_samap',
                    color='segment_celltype',
                    groups=['29_Lineage_Prechordal_Plate',
                            '32_Lineage_Notochord',
                            '59_Lineage_Midbrain_Neural_Crest',
                            '59_Lineage_Placode_Olfactory',
                            '61_unassigned',
                           '70_unassigned'],
                    frameon=True,
               title='Zebrafish '+config["stage_zebra"], ax=axs[2])


### Visualise prechordal plate markers

In [ ]:
zebra_pp_markers=['dr_six3b','dr_six7','dr_gsc','dr_foxa3','dr_otx1a']
amp_pp_markers=['bf_Six3/6','bf_Six4/5','bf_Cerberus','bf_Dmbx','bf_Fz5/8']
print('Amphioxus PP markers')
with rc_context({"figure.figsize": (4, 6)}):
    sc.pl.embedding(sm.sams['bf'].adata,
                    basis='X_umap_samap',
                    color=amp_pp_markers,
                    frameon=False,
                    colorbar_loc=None,
                    ncols=5)
    print('Zebrafish PP markers')
    sc.pl.embedding(sm.sams['dr'].adata,
                    basis='X_umap_samap',
                    color=zebra_pp_markers,
                    frameon=False,
                    colorbar_loc=None,
                    ncols=5)

### Notochord markers

In [ ]:
zebra_noto_markers=['dr_noto','dr_mnx1','dr_foxd5','dr_foxa2','dr_chd']
amp_noto_markers=['bf_Admp','bf_Mnx','bf_Goosecoid','bf_Shh','bf_Chordin']
print('Amphioxus Noto markers')
with rc_context({"figure.figsize": (4, 6)}):
    sc.pl.embedding(sm.sams['bf'].adata,
                    basis='X_umap_samap',
                    color=amp_noto_markers,
                    frameon=False,
                    colorbar_loc=None,
                    ncols=5)
    print('Zebrafish Noto markers')
    sc.pl.embedding(sm.sams['dr'].adata,
                    basis='X_umap_samap',
                    color=zebra_noto_markers,
                    frameon=False,
                    colorbar_loc=None,
                    ncols=5)

## Calculating cell type mapping scores

In [ ]:
D50_fine,MappingTable50_fine = get_mapping_scores(sm,{'dr' : 'segment_celltype', 'bf' : 'celltype_fine'},n_top = 50)
MappingTable50_fine.filter(regex=("bf_.*")).filter(axis='index',regex=("dr_.*")).to_csv('Mapping_table_top50_fine_'+contrast+'.csv')
#D50.head()
D50_coarse,MappingTable50_coarse = get_mapping_scores(sm,{'dr' : 'celltype_coarse', 'bf' : 'celltype_coarse2'},n_top = 50)
MappingTable50_coarse.filter(regex=("bf_.*")).filter(axis='index',regex=("dr_.*")).to_csv('Mapping_table_top50_coarse_'+contrast+'.csv')

## Visualizing mapping scores

### Per celltype

Fine annotation

In [ ]:
plt.gcf().set_size_inches(len(sm.sams['dr'].adata.obs["segment_celltype"].unique())/2,
                         (len(sm.sams['bf'].adata.obs["celltype_fine"].unique())/2)-1)
plt.title('Mapping score fine celltypes '+contrast)
sns.heatmap(MappingTable50_fine.filter(regex=("dr_.*")).filter(axis='index',regex=("bf_.*")),cmap="magma_r",xticklabels=True,yticklabels=True)#,ax=ax)#[0])
plt.savefig('figures/hm_celltype_fine_'+contrast+'.pdf',bbox_inches='tight')
plt.savefig('figures/hm_celltype_fine_'+contrast+'.png',dpi=250,bbox_inches='tight')

Coarse annotation

In [ ]:
plt.gcf().set_size_inches(len(sm.sams['dr'].adata.obs["celltype_coarse"].unique())/2,
                         (len(sm.sams['bf'].adata.obs["celltype_coarse2"].unique())/2)-1)
plt.title('Mapping score coarse celltypes '+contrast)
sns.heatmap(MappingTable50_coarse.filter(regex=("dr_.*")).filter(axis='index',regex=("bf_.*")),cmap="magma_r",xticklabels=True,yticklabels=True)
plt.savefig('figures/hm_celltype_coarse_'+contrast+'.pdf',bbox_inches='tight')
plt.savefig('figures/hm_celltype_coarse_'+contrast+'.png',dpi=250,bbox_inches='tight')

## Others

### Export stacked plots

In [ ]:
ncols = 1
nrows = 3
figsize = 5
wspace = 0.3
fig, axs = plt.subplots(
    nrows=nrows,
    ncols=ncols,
    figsize=(ncols * figsize + figsize * wspace * (ncols - 1), nrows * figsize),
)
plt.subplots_adjust(wspace=wspace)

#axs[0].set_title('SAMap UMAP amphioxus '+config["stage_amp"]+' (red) vs Zebrafish '+config["stage_zebra"]+' (blue)')
sc.pl.embedding(concat_ads,
                    basis='X_umap_samap',
                    color='dataset',
                frameon=True,
                title='Combined by SAMap',
                size=20,
                ax=axs[0],
                palette={'Zebrafish':('#1b9e77',0.8),'Amphioxus':('#d95f02',0.8)},
               show=False)


sc.pl.embedding(sm.sams['bf'].adata,
                    basis='X_umap_samap',
                    color='celltype_coarse2',
                    frameon=True,
                ax=axs[1],
                show=False,
                title='Amphioxus '+config["stage_amp"]+' celltypes',
               size=20,
               palette=pal_bf)

sc.pl.embedding(sm.sams['dr'].adata,
                    basis='X_umap_samap',
                    color='celltype_coarse',
                    frameon=True,
                ax=axs[2],
                show=False,
                title='Zebrafish '+config["stage_zebra"]+' celltypes',
               size=20,
               palette=pal_dr)

#sm.scatter(COLORS={'dr':'#10b9f1','bf':'#f14810'},axes=axs[0],ss={'bf':3,'dr':3},alpha=0.5)

fig.savefig('figures/comparison_'+contrast+'.pdf',bbox_inches='tight')
fig.savefig('figures/comparison_'+contrast+'.png',bbox_inches='tight',dpi=300)

In [ ]:
ncols = 1
nrows = 3
figsize = 5
wspace = 0.3
fig, axs = plt.subplots(
    nrows=nrows,
    ncols=ncols,
    figsize=(ncols * figsize + figsize * wspace * (ncols - 1), nrows * figsize),
)

plt.subplots_adjust(wspace=wspace)

sc.pl.embedding(sm.sams['bf'].adata,
                    basis='X_umap_samap',
                    color='celltype_fine',
                    groups=['PrechordalPlate','Notochord','MNCC-likeCells','PMNCC'],
                    frameon=True,
                   title='Amphioxus '+config["stage_amp"],ax=axs[1], show=False,size=20)
#sm.scatter(COLORS={'dr':'#10b9f1','bf':'#f14810'},axes=axs[1],ss={'bf':5,'dr':5},alpha=0.5)

sc.pl.embedding(concat_ads,
                    basis='X_umap_samap',
                    color='dataset',
                frameon=True,
                title='Combined by SAMap',
                size=20,
                palette={'Zebrafish':('#1b9e77',0.8),'Amphioxus':('#d95f02',0.8)},ax=axs[0],show=False)

sc.pl.embedding(sm.sams['dr'].adata,
                    basis='X_umap_samap',
                    color='segment_celltype',
                    groups=['29_Lineage_Prechordal_Plate',
                            '32_Lineage_Notochord',
                            '59_Lineage_Midbrain_Neural_Crest',
                            '59_Lineage_Placode_Olfactory',
                            '61_unassigned',
                           '70_unassigned'],
                    frameon=True,
                size=20,
               title='Zebrafish '+config["stage_zebra"], ax=axs[2])

fig.savefig('figures/comparison_selection_'+contrast+'.pdf',bbox_inches='tight')
fig.savefig('figures/comparison_selection_'+contrast+'.png',bbox_inches='tight',dpi=300)